In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_values
from sqlalchemy import create_engine
import warnings
import os
import pandas as pd
from fuzzywuzzy import process
import asyncio
import nest_asyncio
from googletrans import Translator


warnings.filterwarnings('ignore')


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

In [134]:
file_path = r'C:\Users\Vignesh\Desktop\SDM\models\data\raw_data.csv'  
sheet_name = 'raw_data'

In [135]:
nest_asyncio.apply()

def translate_sync(text, translator, src='ko', dest='en'):
    """
    Synchronously translate text using the asynchronous translator.
    """
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(translator.translate(text, src=src, dest=dest))
    return result

def read_spreadsheet_with_fuzzy_matching(file_path, sheet_name=None, target_columns=None):
    """
    Read spreadsheet data with fuzzy column name matching, automatically translating
    any Korean column names to English, and stop at the first completely empty row.
    
    Parameters:
        file_path: Path to the spreadsheet file.
        sheet_name: Name of the sheet to read (for Excel files).
        target_columns: List of target column names to match (e.g., ["SCJ-NUMBER", "Name", "BIRTH"]).
        
    Returns:
        pandas DataFrame with selected data.
    """
    file_ext = os.path.splitext(file_path)[1].lower()
    
    # Read file to get raw data and headers
    if file_ext in ['.xlsx', '.xls']:
        raw_data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        header_row = 0  # Assume headers are in the first row
    elif file_ext == '.csv':
        raw_data = pd.read_csv(file_path, header=None)
        header_row = 0
    else:
        raise ValueError(f"Unsupported file format: {file_ext}")
    
    # Find the first completely empty row
    empty_rows = raw_data.isna().all(axis=1)
    if empty_rows.any():
        first_empty_row = empty_rows.idxmax()
        print(f"First completely empty row detected at row {first_empty_row}")
    else:
        first_empty_row = len(raw_data)
        print("No empty rows detected, using all data")
    
    # Re-read the file up to the first empty row
    if file_ext in ['.xlsx', '.xls']:
        df = pd.read_excel(file_path, sheet_name=sheet_name, nrows=first_empty_row - header_row - 1)
    elif file_ext == '.csv':
        df = pd.read_csv(file_path, nrows=first_empty_row - header_row - 1)
    
    # Translate column names (if they contain Korean characters)
    translator = Translator()
    new_columns = {}
    for col in df.columns:
        if any('\uac00' <= ch <= '\ud7a3' for ch in str(col)):
            try:
                result = translate_sync(col, translator, src='ko', dest='en')
                new_columns[col] = result.text
                print(f"Translated '{col}' to '{result.text}'")
            except Exception as e:
                print(f"Error translating column '{col}': {e}. Keeping original.")
                new_columns[col] = col
        else:
            new_columns[col] = col
    df.rename(columns=new_columns, inplace=True)
    df.columns = df.columns.str.lower().str.strip()

    
    # If no target columns specified, return the full DataFrame
    if not target_columns:
        return df
    
    # Fuzzy match the target columns to actual DataFrame columns
    matched_columns = {}
    actual_columns = df.columns.tolist()
    for target in target_columns:
        best_match, score = process.extractOne(target, actual_columns)
        print(f"Matched '{target}' to '{best_match}' with score {score}")
        matched_columns[target] = best_match
    
    # Select only the matched columns
    selected_columns = list(matched_columns.values())
    if len(selected_columns) < len(target_columns):
        print("Warning: Could not find matches for all target columns")
        
    result_df = df[selected_columns]
    
    # Replace forward slashes with dots in the "Name" column (if it exists)
    name_column = matched_columns.get("Name")
    if name_column in result_df.columns:
        result_df[name_column] = result_df[name_column].str.replace('/', '.', regex=False)
    
    return result_df


In [136]:
target_columns = [
    "scj-number", 
    "name", 
    "birth", 
    "phone", 
    "department", 
    "team", 
    "new cell grp", 
    "special note", 
    "office"
]


In [137]:
try:
    # First attempt to read with column names 
    # lower col names
    df = read_spreadsheet_with_fuzzy_matching(file_path, sheet_name, target_columns)
except (ValueError, KeyError) as e:
    print(f"Could not read with column names: {str(e)}")
    


First completely empty row detected at row 842
Translated '직분' to 'Office'
Translated '총등,교등,입교' to 'Total light, school, enrollment'
Translated 'Sunday Service
 '11/26
 정시예배 1
 replay
 recording' to 'Sunday Service
 '11/26
 On -time worship 1
 Replay
 Recording'
Matched 'scj-number' to 'scj-number' with score 100
Matched 'name' to 'name' with score 100
Matched 'birth' to 'birth' with score 100
Matched 'phone' to 'phone no' with score 90
Matched 'department' to 'department' with score 100
Matched 'team' to 'team' with score 100
Matched 'new cell grp' to 'new cell grp' with score 100
Matched 'special note' to 'special note' with score 100
Matched 'office' to 'office' with score 100


In [138]:
print("\n=== Data Overview ===")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
print("\n=== Column Names ===")
print(df.columns.tolist())


=== Data Overview ===
Rows: 841, Columns: 9

=== Column Names ===
['scj-number', 'name', 'birth', 'phone no', 'department', 'team', 'new cell grp', 'special note', 'office']


In [139]:
print("\n=== Data Types and Missing Values ===")
missing_data = pd.DataFrame({
    'Data Type': df.dtypes,
    'Missing Values': df.isnull().sum(),
    'Missing Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
display(missing_data)


=== Data Types and Missing Values ===


,Data Type,Missing Values,Missing Percentage
scj-number,object,0,0.00
name,object,0,0.00
birth,float64,4,0.48
phone no,object,745,88.59
department,object,54,6.42
team,object,24,2.85
new cell grp,object,20,2.38
special note,object,839,99.76
office,object,740,87.99


In [140]:
print("\n=== Data Preview ===")
display(df.head())


=== Data Preview ===


,scj-number,name,birth,phone no,department,team,new cell grp,special note,office
0,00371211-01006,Priscilla Nishi J /,951205.00,NaN,Young adults / Instructor,Educators,Educators,NaN,HQ registration
1,00371211-01029,Theodore Williams /,227.00,NaN,Young adults / Instructor,Educators,Educators,NaN,HQ registration
2,00381126-00718,christopher / John,21027.00,NaN,Young adults,Educators,Educators,NaN,HQ registration
3,00381126-00760,Sophia / Christina,501.00,NaN,Young adults,Educators,Educators,NaN,HQ registration
4,00381126-00764,/ Sofia,10920.00,NaN,Young adults,Educators,Educators,NaN,HQ registration


In [141]:
def add_metadata_columns(df):
    """Add metadata columns for tracking"""
    df['loaded_at'] = datetime.now()
    df['source_file'] = os.path.basename(file_path)
    return df

In [142]:
print("\n=== Cleaning and Transforming Data ===")

# Make a copy of the original data
df_original = df.copy()


=== Cleaning and Transforming Data ===


In [143]:
df = add_metadata_columns(df)
print("Added metadata columns")
df.tail(2)

Added metadata columns


,scj-number,name,birth,phone no,department,team,new cell grp,special note,office,loaded_at,source_file
839,00420123-02088,Sudalaimuthu,20000000.00,91)636-949-2003,Youth,men,ASM 01,NaN,NaN,2025-03-09 12:18:36.205317,raw_data.csv
840,00311213-00174,Rosa,20000000.00,NaN,NaN,NaN,ANY 07,NaN,NaN,2025-03-09 12:18:36.205317,raw_data.csv


In [144]:
print("\n=== Transformation Complete ===")
print(f"New shape: {df.shape}")



=== Transformation Complete ===
New shape: (841, 11)


In [145]:
print("\n=== Loading Data to Supabase ===")

# Database connection parameters
db_params = {
    'host': 'aws-0-ap-southeast-1.pooler.supabase.com',  # Update with your Supabase host
    'port': 5432,
    'dbname': 'postgres',
    'user': 'postgres.lqugerdmxqpvigfchfgz',  # Update with your Supabase user
    'password': 'India144000a##'  # Update with your Supabase password
}


=== Loading Data to Supabase ===


In [146]:
schema_name = 'public'  # Update if needed
table_name = 'staging_data'  # Update with your staging table name
full_table_name = f"{schema_name}.{table_name}"
print(f"Loading data to table '{full_table_name}'")

Loading data to table 'public.staging_data'


In [147]:
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")

# Load data directly with pandas
df.to_sql(
    table_name, 
    engine, 
    schema=schema_name,
    if_exists='replace',  # 'replace' to recreate the table, 'append' to add to existing
    index=False
)

print(f"Data loaded to {full_table_name} using SQLAlchemy")

print("\n=== Summary ===")
print(f"Processed {df.shape[0]} rows with {df.shape[1]} columns")
print(f"Selected columns only: {df.columns.tolist()}")
print(f"Stopped reading at first empty row")
print(f"Data ready for loading to Supabase staging table: {full_table_name}")

Data loaded to public.staging_data using SQLAlchemy

=== Summary ===
Processed 841 rows with 11 columns
Selected columns only: ['scj-number', 'name', 'birth', 'phone no', 'department', 'team', 'new cell grp', 'special note', 'office', 'loaded_at', 'source_file']
Stopped reading at first empty row
Data ready for loading to Supabase staging table: public.staging_data
